In [1]:
!pip install findspark

In [2]:
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SparkContext
from lib.prepro import Preprocessing, SparkDataFrame
from tqdm.notebook import tqdm
import dask.dataframe as dd
import os
from pprint import pprint

In [3]:
PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터/*'
PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터/*'
PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터/*'

# PATH_CCTV_DATA = './cctv_data/*'
# PATH_SAND_DATA = './sand_data/*'
# PATH_SAND_LABEL = './sand_label_data/*'

# PATH_CCTV_DATA = 'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터'
# PATH_SAND_DATA = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/01.원천데이터'
# PATH_SAND_LABEL = 'G:/industry_data/264.건설 모래 품질 관리데이터/01-1.정식개방데이터/Training/02.라벨링데이터'

In [4]:
prepro = Preprocessing()
sdf = SparkDataFrame()

In [5]:
cctv_paths = prepro.get_all_file_paths(PATH_CCTV_DATA)
sand_paths = prepro.get_all_file_paths(PATH_SAND_DATA)
sand_label_paths = prepro.get_all_file_paths(PATH_SAND_LABEL)

In [6]:
cctv_df = sdf.get_spark_dataframe(cctv_paths)
sand_df = sdf.get_spark_dataframe(sand_paths)
sand_label_df = sdf.get_spark_dataframe(sand_label_paths)

In [7]:
cctv_data_info = prepro.print_data_type_num(cctv_paths)
sand_data_info = prepro.print_data_type_num(sand_paths)
sand_label_data_info = prepro.print_data_type_num(sand_label_paths)

In [13]:
cctv_paths[0]

'G:/industry_data/117.산업시설 열화상 CCTV 데이터/01.데이터/1.Training/원천데이터\\TS10\\2.변전소\\4.계기용변압기(PT)\\0204010917(0132)(0497)0301.jpg'

In [12]:
prepro.get_folder_name(cctv_paths[0])

'4.계기용변압기(PT)'

In [8]:
pprint(cctv_data_info)
pprint(sand_data_info)
pprint(sand_label_data_info)

('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 1457336 \n'
 '            png counts: 0 \n'
 '            jpg counts: 1093002 \n'
 '            csv counts: 364334\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 490755 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 0\n'
 '            etc counts: 0 \n'
 '            -------------------------------------- \n'
 '            ')
('\n'
 '            데이터 처리 정보: \n'
 '            -------------------------------------- \n'
 '            전체 이미지 데이터 수: 490755 \n'
 '            png counts: 0 \n'
 '            jpg counts: 0 \n'
 '            csv counts: 0\n'
 '            json counts: 49